In [1]:
import os
import os.path
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
t = 125
thres = 4.5


In [2]:
x = []
y = []
file_cnt = 0
for dirpath, dirnames, filenames in os.walk("../Fall Data (New)/"):
    for filename in [f for f in filenames if f.endswith(".txt")]:
        df = pd.read_csv(os.path.join(dirpath, filename),header=None)
        fall_time = df[df[1].isna()]
        df = df[df[1].notna()]
        ax = np.array(df[1])
        ay = np.array(df[2])
        az = np.array(df[3])
        gx = np.array(df[4])
        gy = np.array(df[5])
        gz = np.array(df[6])
        time = np.array(df[0])
        a = np.sqrt(ax**2 + ay**2 + az**2)
        if filename[0] == "A":
            # ADL file
            file_cnt += 1
            for i in range(a.shape[0]):
                if a[i] >= thres:
                    st = i - t
                    ed =  i + t
                    if st < 0 or ed >= a.shape[0]:
                        continue

                    tmp = np.array(list(zip(ax[st:ed],ay[st:ed],az[st:ed],gx[st:ed],gy[st:ed],gz[st:ed])))


                    x.append(tmp)
                    y.append(1)
                    
        elif filename[11] == "B":
            i = np.argmax(a)
            st = i - t
            ed =  i + t
            if st < 0 or ed >= a.shape[0] or a[i] < thres:
                print(filename,st,ed,a[i])
                continue
            file_cnt += 1
            tmp = np.array(list(zip(ax[st:ed],ay[st:ed],az[st:ed],gx[st:ed],gy[st:ed],gz[st:ed])))
            x.append(tmp)
            y.append(2)
            print("Same floor Fall:",filename)
            continue

        else:
            # Fall Data
            i = np.argmax(a)
            st = i - t
            ed =  i + t
            if st < 0 or ed >= a.shape[0] or a[i] < thres:
                print(filename,st,ed,a[i])
                continue
            file_cnt += 1
            print("Fall:",filename)
            tmp = np.array(list(zip(ax[st:ed],ay[st:ed],az[st:ed],gx[st:ed],gy[st:ed],gz[st:ed])))
            x.append(tmp)
            if len(y) == 1157:
                print(i,filename)
            y.append(0)
            
print(file_cnt)

Fall: TESTING_ML_v10.txt
Fall: TESTING_ML_v15.txt
Fall: TESTING_ML_v16.txt
Fall: TESTING_ML_v19.txt
Fall: TESTING_ML_v20.txt
Fall: TESTING_ML_v21.txt
Fall: TESTING_ML_v22.txt
797 TESTING_ML_v22.txt
Fall: TESTING_ML_v23.txt
Fall: TESTING_ML_v25.txt
Fall: TESTING_ML_v28.txt
Fall: TESTING_ML_v29.txt
Fall: TESTING_ML_v30.txt
Fall: TESTING_ML_v40.txt
Fall: TESTING_ML_v49.txt
Fall: TESTING_ML_v50.txt
Fall: TESTING_ML_v51.txt
Fall: TESTING_ML_v52.txt
Fall: TESTING_ML_v53.txt
Fall: TESTING_ML_v54.txt
Fall: TESTING_ML_v55.txt
Fall: TESTING_ML_v56.txt
Fall: TESTING_ML_v57.txt
Fall: TESTING_ML_v58.txt
Fall: TESTING_ML_v59.txt
Fall: TESTING_ML_v8.txt
Fall: TESTING_ML_v9.txt
Same floor Fall: TESTING_ML_BB_v100.txt
Same floor Fall: TESTING_ML_BB_v101.txt
Same floor Fall: TESTING_ML_BB_v69.txt
Same floor Fall: TESTING_ML_BB_v70.txt
Same floor Fall: TESTING_ML_BB_v71.txt
Same floor Fall: TESTING_ML_BB_v72.txt
Same floor Fall: TESTING_ML_BB_v73.txt
Same floor Fall: TESTING_ML_BB_v74.txt
Same floor Fall

In [3]:
from collections import Counter
cnt =  Counter(y)
cnt

Counter({1: 1151, 0: 26, 2: 27})

In [4]:
X = np.array(x)
Y = np.array(y)
X = X.reshape(-1,t*2,6,1)
print(X.shape,Y.shape)


(1204, 250, 6, 1) (1204,)


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=32)

orig_shape = X_train.shape

wX = X_train.reshape(-1,t*12)
print(X_train.shape,Y.shape,wX.shape)

(963, 250, 6, 1) (1204,) (963, 1500)


In [6]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
over = SMOTE(sampling_strategy='not majority')
steps = [('o', over)]
pipeline = Pipeline(steps=steps)
# transform the dataset
new_train_X, new_train_Y = pipeline.fit_resample(wX, y_train)

In [7]:
new_train_X = new_train_X.reshape((-1, t*2, 6, 1))
print(new_train_X.shape,new_train_Y.shape)

(2763, 250, 6, 1) (2763,)


In [8]:
y_train_cat = keras.utils.to_categorical(new_train_Y)
y_test_cat = keras.utils.to_categorical(y_test)

In [9]:
from collections import Counter
cnt =  Counter(new_train_Y)
cnt

Counter({1: 921, 0: 921, 2: 921})

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
model = Sequential()

model.add(Conv2D(7, kernel_size=(10,6), activation='relu', input_shape=(t*2,6,1),strides=10))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))
adam = tf.keras.optimizers.Adam(learning_rate=0.001,)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 1, 7)          427       
                                                                 
 flatten (Flatten)           (None, 175)               0         
                                                                 
 dense (Dense)               (None, 3)                 528       
                                                                 
Total params: 955
Trainable params: 955
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(new_train_X, y_train_cat, validation_data=(X_test,y_test_cat), epochs=20,batch_size=128)


Epoch 1/20
22/22 [==============================] - 1s 11ms/step - loss: 0.8270 - accuracy: 0.6301 - val_loss: 0.5742 - val_accuracy: 0.8299
Epoch 2/20
22/22 [==============================] - 0s 5ms/step - loss: 0.4255 - accuracy: 0.9135 - val_loss: 0.2199 - val_accuracy: 0.9876
Epoch 3/20
22/22 [==============================] - 0s 6ms/step - loss: 0.2116 - accuracy: 0.9801 - val_loss: 0.1052 - val_accuracy: 0.9876
Epoch 4/20
22/22 [==============================] - 0s 5ms/step - loss: 0.1110 - accuracy: 0.9946 - val_loss: 0.0679 - val_accuracy: 0.9917
Epoch 5/20
22/22 [==============================] - 0s 5ms/step - loss: 0.0661 - accuracy: 0.9975 - val_loss: 0.0559 - val_accuracy: 0.9917
Epoch 6/20
22/22 [==============================] - 0s 5ms/step - loss: 0.0433 - accuracy: 0.9971 - val_loss: 0.0496 - val_accuracy: 0.9917
Epoch 7/20
22/22 [==============================] - 0s 6ms/step - loss: 0.0305 - accuracy: 0.9978 - val_loss: 0.0428 - val_accuracy: 0.9959
Epoch 8/20
22/22 [=

In [12]:
results = model.evaluate(X_test, y_test_cat, batch_size=128)
print(X_test.shape[0])

2/2 [==============================] - 0s 3ms/step - loss: 0.0361 - accuracy: 0.9959
241


In [13]:
from sklearn.metrics import confusion_matrix

predictions = model.predict(X, batch_size=128, verbose=0)
classes_x=(np.argmax(predictions,axis=1))
true_classes = Y

c = confusion_matrix(Y, classes_x)
print('Confusion matrix:\n', c)
print('Accuracy:{}%'.format((c[0, 0] + c[1, 1]) * 100 / (c[0, 1] + c[0, 0] + c[1, 1] + c[1, 0])))
print('Sensitivity:{}%'.format(c[0, 0]*100 / (c[0, 1] + c[0, 0])))
print('Specificity:{}%'.format(c[1, 1]*100 / (c[1, 1] + c[1, 0])))


Confusion matrix:
 [[  25    1    0]
 [   0 1151    0]
 [   0    0   27]]
Accuracy:99.91503823279524%
Sensitivity:96.15384615384616%
Specificity:100.0%


In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()


# Save the model to disk
open('../model/model.tflite', "wb").write(tflite_model)
model.save('../model')
# !xxd -i converted_model.tflite > model_data.cc

INFO:tensorflow:Assets written to: C:\Users\soods\AppData\Local\Temp\tmptz4kfgjw\assets


INFO:tensorflow:Assets written to: C:\Users\soods\AppData\Local\Temp\tmptz4kfgjw\assets


INFO:tensorflow:Assets written to: ../model\assets


INFO:tensorflow:Assets written to: ../model\assets


In [18]:
from tinymlgen import port
open('../model/model.h', "w").write((port(model, optimize=False)))/1024

INFO:tensorflow:Assets written to: C:\Users\soods\AppData\Local\Temp\tmp785916k9\assets


INFO:tensorflow:Assets written to: C:\Users\soods\AppData\Local\Temp\tmp785916k9\assets


35462